In [ ]:
from frontend import App

app = App("cards", renew=True)

mesh_res, n_stack, card_height = 8, 8, 0.25
card_width = 0.75 * card_height
V, F = app.mesh.rectangle(mesh_res,card_width,card_height,[0,0,1],[0,1,0])
app.asset.add.tri("card", V, F)

V, F, T = app.mesh.icosphere(r=0.15,subdiv_count=3).tetrahedralize()
app.asset.add.tet("sphere", V, F, T)

scene = app.scene.create("house-of-cards")
param = app.session.param()
angle, gap = 25.0, param.get("contact-ghat")

def make_row(n: int, _x: float, y: float) -> float:
    x, _y, ceil_x = 0, 0, []
    for i in range(n):
        left = scene.add("card").rotate(-angle,"z")
        left.at(x-left.min(0)+(_x if i==0 else 0),y-left.min(1),0)
        if i == 0:
            _x = left.max(0)
        right = scene.add("card").rotate(angle,"z")
        shift = gap + left.max(0)-right.min(0)
        right.at(shift,y-right.min(1),0)
        if i < n-1:
            ceil_x.append(right.max(0))
        x = right.max(0) + gap
        max_y = right.max(1) + gap
    for i, x in enumerate(ceil_x):
        z = max_y if i%2 == 0 else max_y+gap
        ceil = scene.add("card").rotate(-90,"z").at(x,z,0)
        _y = max(_y,ceil.max(1))
    return _x, _y + gap

_x, _y = -0.75, gap
for i in reversed(range(n_stack)):
    _x, _y = make_row(i+1, _x, _y)

scene.add("sphere").at(-2,1,0).velocity(2.3,0,0)
scene.add_invisible_wall([0,0,0],[0,1,0])

fixed = scene.build().report()
fixed.preview();

In [ ]:
param.set("friction",0.5);
param.set("area-young-mod",30000);
param.set("bend",1e6).set("dt",0.01).set("min-newton-steps",32);
session = app.session.create("8-stairs").init(fixed)
session.start(param).preview();
session.stream();

In [ ]:
# run this cell after sufficnt frames are simulated
session.animate();

In [ ]:
# this is for CI
assert session.finished()

In [ ]:
# export all simulated frames
session.export.animation(f"export/{scene.info.name}/{session.info.name}")